In [1]:
# =========================
# 🧼 Enhanced E-commerce Data Cleaning Pipeline (Olist Dataset)
# =========================

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from ydata_profiling import ProfileReport

# === Paths ===
RAW_PRODUCTS = "raw_data/olist_products_dataset.csv"
RAW_ORDERS = "raw_data/olist_order_items_dataset.csv"
TRANSLATIONS = "raw_data/product_category_name_translation.csv"
CLEANED_PATH = "cleaned_data/cleaned_products.csv"
VISUALS_PATH = "visuals/price_distribution.png"
REPORT_PATH = "visuals/olist_product_report.html"

# === Create folders if not exist ===
os.makedirs("raw_data", exist_ok=True)
os.makedirs("cleaned_data", exist_ok=True)
os.makedirs("visuals", exist_ok=True)

# === Load Datasets ===
print("🔹 Loading data...")
products_df = pd.read_csv(RAW_PRODUCTS)
orders_df = pd.read_csv(RAW_ORDERS)

print("✅ Products shape:", products_df.shape)
print("✅ Orders shape:", orders_df.shape)

# === Merge product and price info ===
merged_df = pd.merge(
    products_df, 
    orders_df[['product_id', 'price']], 
    on='product_id', 
    how='left'
)

# === Step 1: Drop Duplicates ===
merged_df = merged_df.drop_duplicates()

# === Step 2: Handle Missing Values ===
# Fill missing prices with median of the product's category
merged_df['price'] = merged_df.groupby('product_category_name')['price'].transform(
    lambda x: x.fillna(x.median())
)
# If still missing (entire category missing), fill with global median
merged_df['price'] = merged_df['price'].fillna(merged_df['price'].median())

# Drop rows with missing category names
merged_df = merged_df.dropna(subset=['product_category_name'])

# === Step 3: Normalize Strings ===
merged_df['product_category_name'] = (
    merged_df['product_category_name']
    .str.lower()
    .str.strip()
    .str.replace(' ', '_')
)

# === Step 4: Add English Translations ===
if os.path.exists(TRANSLATIONS):
    translations = pd.read_csv(TRANSLATIONS)
    merged_df = pd.merge(
        merged_df, 
        translations, 
        how='left', 
        on='product_category_name'
    )
    print("✅ Added English category translations")

# === Step 5: Feature Engineering ===
# Price Buckets
price_bins = [0, 20, 50, 100, 200, 500, float('inf')]
price_labels = ['<20', '20-50', '50-100', '100-200', '200-500', '500+']

merged_df['price_bucket'] = pd.cut(
    merged_df['price'],
    bins=price_bins,
    labels=price_labels
)

# Category Popularity
category_counts = merged_df['product_category_name'].value_counts()
merged_df['category_popularity'] = (
    merged_df['product_category_name']
    .map(category_counts)
    .astype('int')
)

# Product Dimensions (if available)
dimension_cols = ['product_length_cm', 'product_height_cm', 'product_width_cm']
if all(col in merged_df.columns for col in dimension_cols):
    merged_df['product_volume_cm3'] = (
        merged_df['product_length_cm'] * 
        merged_df['product_height_cm'] * 
        merged_df['product_width_cm']
    )

# === Step 6: Outlier Removal ===
Q1 = merged_df['price'].quantile(0.05)  # Using 5th percentile for more conservative cutoff
Q3 = merged_df['price'].quantile(0.95)
IQR = Q3 - Q1
merged_df = merged_df[~((merged_df['price'] < (Q1 - 1.5 * IQR)) | 
                      (merged_df['price'] > (Q3 + 1.5 * IQR)))]

# === Step 7: Save Cleaned Data ===
merged_df.to_csv(CLEANED_PATH, index=False)
print(f"✅ Cleaned data saved to {CLEANED_PATH}")

# === Step 8: Generate Visualizations ===
plt.figure(figsize=(12, 6))
sns.histplot(merged_df['price'], bins=50, kde=True, color='skyblue')
plt.title("Price Distribution After Cleaning", fontsize=14)
plt.xlabel("Price (R$)", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.tight_layout()
plt.savefig(VISUALS_PATH, dpi=300)
plt.close()  # Close the plot to free memory
print(f"📊 Visualization saved to {VISUALS_PATH}")

# Top Categories by Product Count
if 'product_category_name_english' in merged_df.columns:
    plt.figure(figsize=(12, 8))
    top_cats = merged_df['product_category_name_english'].value_counts().head(10)
    sns.barplot(x=top_cats.values, y=top_cats.index, palette='viridis')
    plt.title("Top 10 Product Categories by Count", fontsize=14)
    plt.xlabel("Count", fontsize=12)
    plt.ylabel("Category", fontsize=12)
    plt.tight_layout()
    plt.savefig("visuals/top_categories.png", dpi=300)
    plt.close()

# === Step 9: Generate EDA Report ===
profile = ProfileReport(
    merged_df, 
    title="Olist Product Report", 
    explorative=True,
    minimal=True
)
profile.to_file(REPORT_PATH)
print(f"📄 EDA report saved to {REPORT_PATH}")

# === Step 10: Summary Statistics ===
print("\n📊 Summary Statistics:")
print(f"Total products: {len(merged_df)}")
print(f"Unique categories: {merged_df['product_category_name'].nunique()}")
print(f"Median price: R${merged_df['price'].median():.2f}")
print(f"Average price: R${merged_df['price'].mean():.2f}")
print("\nPrice Bucket Distribution:")
print(merged_df['price_bucket'].value_counts().sort_index())

🔹 Loading data...
✅ Products shape: (32951, 9)
✅ Orders shape: (112650, 7)
✅ Added English category translations
✅ Cleaned data saved to cleaned_data/cleaned_products.csv
📊 Visualization saved to visuals/price_distribution.png


C:\Users\RAJUA\AppData\Local\Temp\ipykernel_29840\1407826757.py:128: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=top_cats.values, y=top_cats.index, palette='viridis')


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:00<00:00, 42.94it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

📄 EDA report saved to visuals/olist_product_report.html

📊 Summary Statistics:
Total products: 41316
Unique categories: 73
Median price: R$80.00
Average price: R$130.28

Price Bucket Distribution:
price_bucket
<20         3427
20-50      10081
50-100     11012
100-200    10261
200-500     5002
500+        1533
Name: count, dtype: int64


In [2]:
# Top 10 most popular categories (by product count)
top_categories = merged_df['product_category_name_english'].value_counts().head(10)
print("Top 10 Most Popular Categories (by Product Count):")
print(top_categories)

# Average price per category
avg_price_per_category = merged_df.groupby('product_category_name_english')['price'].mean().sort_values(ascending=False)
print("\nAverage Price per Category:")
print(avg_price_per_category)

# Price variation across buckets
price_bucket_variation = merged_df.groupby('price_bucket')['price'].agg(['mean', 'std'])
print("\nPrice Variation Across Buckets:")
print(price_bucket_variation)

Top 10 Most Popular Categories (by Product Count):
product_category_name_english
bed_bath_table           3748
sports_leisure           3553
health_beauty            3190
furniture_decor          3067
housewares               2909
computers_accessories    2314
auto                     2186
watches_gifts            1993
toys                     1836
telephony                1516
Name: count, dtype: int64

Average Price per Category:
product_category_name_english
computers                                697.844138
home_appliances_2                        338.812719
agro_industry_and_commerce               279.522360
watches_gifts                            265.853116
small_appliances_home_oven_and_coffee    263.178000
                                            ...    
cds_dvds_musicals                         55.000000
food_drink                                53.616899
diapers_and_hygiene                       52.972667
dvds_blu_ray                              52.419200
flowers       

C:\Users\RAJUA\AppData\Local\Temp\ipykernel_29840\3312925934.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_bucket_variation = merged_df.groupby('price_bucket')['price'].agg(['mean', 'std'])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# === Prepare Data ===
# Select features and target
features = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 
            'product_weight_g', 'product_volume_cm3', 'category_popularity']
target = 'price_bucket'

# Drop rows with missing values in selected features or target
ml_data = merged_df[features + [target]].dropna()

# Encode target variable (price_bucket)
label_encoder = LabelEncoder()
ml_data[target] = label_encoder.fit_transform(ml_data[target])

# Split data into train and test sets
X = ml_data[features]
y = ml_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Train Model ===
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# === Evaluate Model ===
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [9]:
import streamlit as st
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt


    # Load the cleaned data
CLEANED_PATH = 'cleaned_data/cleaned_products.csv'
merged_df = pd.read_csv(CLEANED_PATH)
top_cats = merged_df['product_category_name_english'].value_counts().head(10)
avg_price_per_category = merged_df.groupby('product_category_name_english')['price'].mean().sort_values(ascending=False)
top_categories = merged_df['product_category_name_english'].value_counts().head(10)
price_bucket_variation = merged_df.groupby('price_bucket')['price'].agg(['mean', 'std'])




# === Prepare Data ===
# Select features and target


# === Streamlit App ===
st.title("Olist E-commerce Dashboard")

# Sidebar
st.sidebar.header("Navigation")
page = st.sidebar.selectbox("Select a Page", ["Overview", "Category Insights", "Price Analysis", "Model Performance"])

# Overview Page
if page == "Overview":
    st.header("Overview")
    st.write("### Cleaned Data Summary")
    st.write(merged_df.describe())
    st.write("### Top 10 Categories by Product Count")
    st.bar_chart(top_cats)

# Category Insights Page
elif page == "Category Insights":
    st.header("Category Insights")
    st.write("### Average Price per Category")
    st.bar_chart(avg_price_per_category)
    st.write("### Top 10 Most Popular Categories")
    st.bar_chart(top_categories)

# Price Analysis Page
elif page == "Price Analysis":
    st.header("Price Analysis")
    st.write("### Price Distribution")
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(merged_df['price'], bins=50, kde=True, color='skyblue', ax=ax)
    ax.set_title("Price Distribution After Cleaning")
    ax.set_xlabel("Price (R$)")
    ax.set_ylabel("Count")
    st.pyplot(fig)

    st.write("### Price Variation Across Buckets")
    st.write(price_bucket_variation)

# Model Performance Page
elif page == "Model Performance":
    st.header("Model Performance")
    st.write("### Classification Report")
    st.text(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
    st.write("### Accuracy")
    st.metric("Accuracy", f"{accuracy_score(y_test, y_pred):.2%}")

    st.write("### Feature Importance")
    feature_importance = pd.DataFrame({
        "Feature": features,
        "Importance": model.feature_importances_
    }).sort_values(by="Importance", ascending=False)
    st.bar_chart(feature_importance.set_index("Feature"))

# Run the app with `streamlit run app.py`

In [11]:
!pip install shap install shap

ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for install
